# OpenAI Realtime API - Arabic Language Testing
Set up the openai key in the secrets section in colab as OPENAI_API_KEY and enable notebook access.

In [1]:
# Install required packages
!pip install -q openai websockets

In [2]:
import os
import json
import base64
import asyncio
import websockets
from IPython.display import Audio, display
import io

In [ ]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [5]:
# ==========================================
# 1. ARABIC SPEECH-TO-TEXT TEST
# ==========================================

async def test_arabic_speech_to_text(audio_file_path):
    """
    Test Arabic speech-to-text using OpenAI Realtime API
    """
    print("🎤 Testing Arabic Speech-to-Text...\n")

    url = "wss://api.openai.com/v1/realtime?model=gpt-realtime"

    async with websockets.connect(
        url,
        additional_headers={
            "Authorization": f"Bearer {OPENAI_API_KEY}",
            "OpenAI-Beta": "realtime=v1"
        }
    ) as ws:
        # Configure session
        await ws.send(json.dumps({
            "type": "session.update",
            "session": {
                "modalities": ["text", "audio"],
                "instructions": "You are a helpful assistant that understands Arabic.",
                "voice": "alloy",
                "input_audio_format": "pcm16",
                "output_audio_format": "pcm16",
                "input_audio_transcription": {
                    "model": "whisper-1"
                }
            }
        }))

        # Read and send audio file
        with open(audio_file_path, 'rb') as f:
            audio_data = f.read()
            audio_base64 = base64.b64encode(audio_data).decode('utf-8')

        await ws.send(json.dumps({
            "type": "input_audio_buffer.append",
            "audio": audio_base64
        }))

        await ws.send(json.dumps({
            "type": "input_audio_buffer.commit"
        }))

        # Listen for transcription
        transcription = ""
        async for message in ws:
            data = json.loads(message)
            if data.get("type") == "conversation.item.input_audio_transcription.completed":
                transcription = data.get("transcript", "")
                print(f"✅ Transcription: {transcription}\n")
                break
            elif data.get("type") == "error":
                print(f"❌ Error: {data}")
                break

        return transcription

# ==========================================
# 2. ARABIC TEXT-TO-SPEECH TEST
# ==========================================

async def test_arabic_text_to_speech(arabic_text):
    """
    Test Arabic text-to-speech using OpenAI Realtime API
    """
    print(f"🔊 Testing Arabic Text-to-Speech...\n")
    print(f"Input text: {arabic_text}\n")

    url = "wss://api.openai.com/v1/realtime?model=gpt-realtime"

    async with websockets.connect(
        url,
        additional_headers={
            "Authorization": f"Bearer {OPENAI_API_KEY}",
            "OpenAI-Beta": "realtime=v1"
        }
    ) as ws:
        # Wait for session.created
        session_ready = False
        while not session_ready:
            msg = await ws.recv()
            data = json.loads(msg)
            if data.get("type") == "session.created":
                session_ready = True
                print("✓ Session created")

        # Configure session
        await ws.send(json.dumps({
            "type": "session.update",
            "session": {
                "modalities": ["text", "audio"],
                "instructions": "You are a helpful assistant. Speak clearly in Arabic.",
                "voice": "alloy",
                "output_audio_format": "pcm16",
                "turn_detection": None  # Disable turn detection for text input
            }
        }))

        # Wait for session.updated
        msg = await ws.recv()
        data = json.loads(msg)
        if data.get("type") == "session.updated":
            print("✓ Session configured")

        # Create conversation item with the text
        await ws.send(json.dumps({
            "type": "conversation.item.create",
            "item": {
                "type": "message",
                "role": "user",
                "content": [
                    {
                        "type": "input_text",
                        "text": arabic_text
                    }
                ]
            }
        }))

        # Request response
        await ws.send(json.dumps({
            "type": "response.create",
            "response": {
                "modalities": ["audio", "text"]
            }
        }))

        # Collect audio chunks and text
        audio_chunks = []
        response_text = ""

        async for message in ws:
            data = json.loads(message)
            msg_type = data.get("type")

            if msg_type == "response.audio.delta":
                audio_base64 = data.get("delta", "")
                audio_chunks.append(base64.b64decode(audio_base64))
                print(".", end="", flush=True)

            elif msg_type == "response.audio_transcript.delta":
                text_delta = data.get("delta", "")
                response_text += text_delta
                print(text_delta, end="", flush=True)

            elif msg_type == "response.output_item.done":
                item = data.get("item", {})
                content = item.get("content", [])
                for c in content:
                    if c.get("type") == "text":
                        text = c.get("text", "")
                        if text:
                            print(f"\n📝 Text response: {text}")
                            response_text = text

            elif msg_type == "response.done":
                print(f"\n✅ Audio generation completed\n")
                break

            elif msg_type == "error":
                print(f"❌ Error: {data}")
                break

        # Combine and save audio
        if audio_chunks:
            audio_data = b''.join(audio_chunks)

            # Create WAV file manually
            import struct
            sample_rate = 24000
            num_channels = 1
            bits_per_sample = 16

            # WAV file header
            byte_rate = sample_rate * num_channels * bits_per_sample // 8
            block_align = num_channels * bits_per_sample // 8
            data_size = len(audio_data)

            wav_header = struct.pack('<4sI4s4sIHHIIHH4sI',
                b'RIFF',
                36 + data_size,
                b'WAVE',
                b'fmt ',
                16,  # fmt chunk size
                1,   # PCM
                num_channels,
                sample_rate,
                byte_rate,
                block_align,
                bits_per_sample,
                b'data',
                data_size
            )

            wav_data = wav_header + audio_data
            audio_file = "arabic_tts_output.wav"

            with open(audio_file, 'wb') as f:
                f.write(wav_data)

            print(f"💾 Audio saved to: {audio_file}")
            display(Audio(audio_file, autoplay=True))
            return audio_file

        return None

# ==========================================
# 3. ARABIC QUESTION-ANSWER TEST
# ==========================================

async def test_arabic_qa(question):
    """
    Test Arabic question-answering using OpenAI Realtime API
    """
    print(f"💬 Testing Arabic Q&A...\n")
    print(f"Question: {question}\n")

    url = "wss://api.openai.com/v1/realtime?model=gpt-realtime"

    async with websockets.connect(
        url,
        additional_headers={
            "Authorization": f"Bearer {OPENAI_API_KEY}",
            "OpenAI-Beta": "realtime=v1"
        }
    ) as ws:
        # Wait for session.created
        session_ready = False
        while not session_ready:
            msg = await ws.recv()
            data = json.loads(msg)
            if data.get("type") == "session.created":
                session_ready = True

        # Configure session
        await ws.send(json.dumps({
            "type": "session.update",
            "session": {
                "modalities": ["text"],
                "instructions": "أنت مساعد مفيد. أجب على جميع الأسئلة باللغة العربية.",  # You are a helpful assistant. Answer all questions in Arabic.
                "turn_detection": None
            }
        }))

        # Wait for session.updated
        msg = await ws.recv()

        # Send question
        await ws.send(json.dumps({
            "type": "conversation.item.create",
            "item": {
                "type": "message",
                "role": "user",
                "content": [
                    {
                        "type": "input_text",
                        "text": question
                    }
                ]
            }
        }))

        # Request response
        await ws.send(json.dumps({
            "type": "response.create",
            "response": {
                "modalities": ["text"]
            }
        }))

        # Collect response
        answer = ""

        async for message in ws:
            data = json.loads(message)
            msg_type = data.get("type")

            if msg_type == "response.text.delta":
                delta = data.get("delta", "")
                answer += delta
                print(delta, end="", flush=True)

            elif msg_type == "response.audio_transcript.delta":
                delta = data.get("delta", "")
                answer += delta
                print(delta, end="", flush=True)

            elif msg_type == "response.output_item.done":
                item = data.get("item", {})
                content = item.get("content", [])
                for c in content:
                    if c.get("type") == "text":
                        text = c.get("text", "")
                        if text and not answer:
                            print(text)
                            answer = text

            elif msg_type == "response.done":
                if answer:
                    print(f"\n\n✅ Response completed\n")
                else:
                    print(f"\n⚠️ Response completed but no text captured\n")
                break

            elif msg_type == "error":
                print(f"❌ Error: {data}")
                break

        return answer

# ==========================================
# MAIN EXECUTION
# ==========================================

async def main():
    print("=" * 50)
    print("🚀 OpenAI Realtime API - Arabic Testing")
    print("=" * 50 + "\n")

    # TEST 1: Speech-to-Text (requires an audio file)
    print("\n" + "=" * 50)
    print("TEST 1: Arabic Speech-to-Text")
    print("=" * 50)
    print("⚠️ Note: You need to upload an Arabic audio file to test this.")
    print("Upload a .wav file and update the path below.\n")

    # Uncomment and update path when you have an audio file
    audio_file = "/content/arabic.wav"
    transcription = await test_arabic_speech_to_text(audio_file)

    # TEST 2: Text-to-Speech
    print("\n" + "=" * 50)
    print("TEST 2: Arabic Text-to-Speech")
    print("=" * 50 + "\n")

    arabic_text = "مرحباً، كيف حالك اليوم؟"  # Hello, how are you today?
    await test_arabic_text_to_speech(arabic_text)

    # # TEST 3: Question-Answer
    print("\n" + "=" * 50)
    print("TEST 3: Arabic Question-Answer")
    print("=" * 50 + "\n")

    questions = [
        "ما هي عاصمة مصر؟",  # What is the capital of Egypt?
        "اشرح لي ما هو الذكاء الاصطناعي",  # Explain to me what artificial intelligence is
        "ما هي فوائد تعلم اللغات الجديدة؟"  # What are the benefits of learning new languages?
    ]

    for q in questions:
        answer = await test_arabic_qa(q)
        await asyncio.sleep(1)

    print("\n" + "=" * 50)
    print("✅ All tests completed!")
    print("=" * 50)

# Run the tests
await main()

print("\n📖 See the testing guide above for detailed verification steps!")

🚀 OpenAI Realtime API - Arabic Testing


TEST 1: Arabic Speech-to-Text
⚠️ Note: You need to upload an Arabic audio file to test this.
Upload a .wav file and update the path below.

🎤 Testing Arabic Speech-to-Text...

✅ Transcription: تقع الطاهرة على جوانب جزر نهر النيل في شمال مصر


TEST 2: Arabic Text-to-Speech

🔊 Testing Arabic Text-to-Speech...

Input text: مرحباً، كيف حالك اليوم؟

✓ Session created
✓ Session configured
مرحباً!... أنا بخير والح..مد لله،.. وأت.منى أن تكون بخير ك......ذلك. كيف أقدر أساعدك اليوم؟.............
✅ Audio generation completed

💾 Audio saved to: arabic_tts_output.wav



TEST 3: Arabic Question-Answer

💬 Testing Arabic Q&A...

Question: ما هي عاصمة مصر؟

عاصمة مصر هي القاهرة.

✅ Response completed

💬 Testing Arabic Q&A...

Question: اشرح لي ما هو الذكاء الاصطناعي

بالتأكيد! الذكاء الاصطناعي هو مجال من مجالات علم الحاسوب يركز على تطوير الأنظمة والبرمجيات التي تمتلك القدرة على محاكاة القدرات الذهنية البشرية. يعني ذلك أن الذكاء الاصطناعي يمكنه التعلم من البيانات، وتحليلها، واتخاذ قرارات أو تنفيذ مهام بطريقة تشبه ما يمكن أن يقوم به الإنسان. 

هناك أنواع مختلفة من الذكاء الاصطناعي، منها الذكاء الاصطناعي الضيق، الذي يركز على أداء مهمة محددة مثل التعرف على الصور أو الترجمة الآلية، والذكاء الاصطناعي العام، الذي يطمح إلى محاكاة القدرات الفكرية للإنسان بشكل شامل. 

التقنيات المستخدمة في الذكاء الاصطناعي تشمل تعلم الآلة (Machine Learning)، حيث يتعلم النظام من البيانات دون أن تتم برمجته بشكل صريح، والشبكات العصبية الاصطناعية، وهي مستوحاة من طريقة عمل الدماغ البشري، بالإضافة إلى خوارزميات معالجة اللغة الطبيعية والرؤية الحاسوبية. 

بفضل الذكاء الاصطناعي، أصبح بالإم